In [1]:
import os
import sys
from pathlib import Path

import pendulum
import numpy as np
import porepy as pp


sys.path.extend([
    '/home/haakon/mastersproject', 
    '/home/haakon/mastersproject/src/mastersproject'
])

import GTS as gts
import GTS.test.util as test_util


/home/haakon/porepy/src/porepy/viz/exporter.py:928: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  angle = np.arccos(np.dot(normals[:, fi], reference))


In [2]:
def setup_simple_gb(**kwargs):
    """ Setup simple grid bucket
    
    Keyword arguments used for
        length_scale,
        scalar_scale,
        path_tail (Default: "test_1"),
        setup_loggers (Default: False),
        
    """
    length_scale = kwargs.get("length_scale", 1)
    scalar_scale = kwargs.get("scalar_scale", 1)
    
    # Storage path details
    path_tail = kwargs.get("path_tail", "test_1")
    name = "test_parameter_scaling_on_regular_grid_ipynb"
    date = pendulum.now().format("YYMMDD")
    _folder_root = f"{name}/{date}/{path_tail}/ss{scalar_scale}ls{length_scale}"
    
    # Parameter setup
    
    # Size of domain - scaled by length scale.
    physdims = np.array([10, 10, 10]) / length_scale

    params = {
            "mesh_args":
                {None},
            "bounding_box":
                {"xmin": 0, "xmax": physdims[0],
                 "ymin": 0, "ymax": physdims[1],
                 "zmin": 0, "zmax": physdims[2],},
            "shearzone_names":
                ["S1_1"],
            "length_scale":
                length_scale,
            "scalar_scale":
                scalar_scale,
        }
    
    setup_loggers = kwargs.get("setup_loggers", False)

    setup = test_util.prepare_setup(
            model=gts.ContactMechanicsBiotISC,
            path_head=_folder_root,
            params=params,
            prepare_simulation=False,
            setup_loggers=setup_loggers,
        )
    
    ### Set up simple grid bucket with one fracture ###
    
    # Grid is 2x2x2 cells, with a fracture in the lower middle part of the grid.
    #  _ _
    # |   |
    # |_|_|

    # Frac point coordinates, scaled by length scale
    f_1 = np.array([
        [5, 5, 5, 5],
        [0, 0, 5, 5],
        [0, 5, 5, 0]

    ]) / length_scale
    
    # Number of cells in each dimension.
    n_cells = np.array([2, 2, 2])

    gb = pp.meshing.cart_grid(
        fracs=[f_1],
        nx=n_cells,
        physdims=physdims,
    )
    
    # Assign the simple grid to the grid bucket
    setup.set_grid(gb)
    
    run_model = kwargs.get("run_model", True)
    if run_model:
        nl_options = {}
        pp.run_time_dependent_model(setup, nl_options)
    
    return setup

# Run with unscaled variables
I.e., the following (default) scalings apply:\
`length_scale = 1`\
`scalar_scale = 1`

In [3]:
unscl = setup_simple_gb()

No injection cell was tagged.
/home/haakon/anaconda3/envs/porepy-safe/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \
/home/haakon/anaconda3/envs/porepy-safe/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


# Next, re-run with scaled variables
We apply the following scalings:\
`length_scale = 1`\
`scalar_scale = 1 * pp.MEGA`

This isolates testing for the `scalar_scale`.

In [4]:
scl = setup_simple_gb(
    length_scale=1,
    scalar_scale=1*pp.MEGA,
)

/home/haakon/anaconda3/envs/porepy-safe/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \
/home/haakon/anaconda3/envs/porepy-safe/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


# Compare the results

Recall, we only scale the pressure scale here

In [6]:
def assert_solutions(scl, unscl, dim):
    """ Assert that solutions scale correctly
    
    Parameters
    ----------
    scl, unscl : ContactMechanicsBiotISC
        scaled and unscaled setups, respectively
    dim : int
        which grid dimension to extract solutions over
    """
    
    # Fetch the scalings
    ls = scl.length_scale
    ss = scl.scalar_scale
    
    # Grids
    gb_scl = scl.gb
    g_scl = gb_scl.grids_of_dimension(dim)[0]

    gb_unscl = unscl.gb
    g_unscl = gb_unscl.grids_of_dimension(dim)[0]

    # Data
    d_s = gb_scl.node_props(g_scl)
    d_u = gb_unscl.node_props(g_unscl)

    state_s = d_s['state']
    state_u = d_u['state']

    ### Assertions ###
    # Assert that each geometric index corresponds to one another
    assert np.allclose(
        g_scl.cell_centers * ls, 
        g_unscl.cell_centers,
    )
    
    ## Non-zero on all grids ##
    # Assert that the pressure results are equal
    assert np.allclose(
        state_s['p'] * ss,
        state_u['p'],
    )
    assert np.allclose(  # Assert that the exported results are upscaled correctly
        state_s['p_exp'],
        state_u['p_exp'],
    )
    
    ## Relevant on matrix (Nd-grids) ##
    # Assert that the displacement results are equal
    if 'u' in state_s:
        assert np.allclose(
            state_s['u'] * ls,
            state_u['u'],
        )
    assert np.allclose(  # Assert that the exported results are upscaled correctly
        state_s['u_exp'],
        state_u['u_exp'],
    )
    
    ## Relevant on fractures (Nd-1 grids)
    
    # Normal and tangential fracture displacement (already upscaled)
    assert np.allclose(
        state_s['normal_frac_u'],
        state_u['normal_frac_u'],
    )
    assert np.allclose(
        state_s['tangential_frac_u'],
        state_u['tangential_frac_u'],
    )
    
    # Traction
    assert np.allclose(
        state_s['traction_exp'],
        state_u['traction_exp'],
    )  
    

In [7]:
## Assert solutions on 3D grid ##
assert_solutions(scl, unscl, 3)

## Assert solutions on 2D grid ##
assert_solutions(scl, unscl, 2)

AssertionError: 

In [8]:
%debug

> <ipython-input-6-497829c817a1>(57)assert_solutions()
     55     assert np.allclose(  # Assert that the exported results are upscaled correctly
     56         state_s['u_exp'],
---> 57         state_u['u_exp'],
     58     )
     59 

ipdb> state_s['u_exp']
array([[0.00024465],
       [0.00010413],
       [0.        ]])
ipdb> state_u['u_exp']
array([[0.00019639],
       [0.00017925],
       [0.        ]])
ipdb> q


In [ ]:
%debug

> <ipython-input-6-497829c817a1>(57)assert_solutions()
     55     assert np.allclose(  # Assert that the exported results are upscaled correctly
     56         state_s['u_exp'],
---> 57         state_u['u_exp'],
     58     )
     59 

ipdb> state_s['u']
*** KeyError: 'u'


In [ ]:
def assert_parameter_scaling(scl, unscl):
    """ Check that all equation parameters are scaled correctly"""
    
    